# How to present CV with Neptune

## Introduction

When training models with cross-validation, you can use Neptune namespaces to organize, visualize and compare models.

By the end of this guide, you will learn how to organize your run to track cross-validation metadata, so that you can easily analyze the results.

[See this example in Neptune](https://app.neptune.ai/o/common/org/showroom/e/SHOW-27624)

<center>
    <a href="https://app.neptune.ai/o/common/org/showroom/e/SHOW-27624">
        <img src="https://neptune.ai/wp-content/uploads/metadata-in-neptune.png" alt="Drawing" style="height: 600px;">
    </a>
</center>
<center><small>CV results presented in Neptune UI</small></center>



## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects) 
<br><br>
* This example assumes: 
    * Basic familiarity with Neptune ([create run and log metadata to it](https://docs.neptune.ai/you-should-know/what-can-you-log-and-display)),
    * Familiarity with cross-validation techniques in machine learning.




## Install Neptune and dependencies

In [ ]:
! pip install -U neptune-client scikit-learn torch torchvision tqdm

## Start a run

To connect your script to Neptune and create a new run, we tell Neptune:
* **Who you are** - with a Neptune API token
* **Where to send your data** - to a Neptune project

The cell below lets you record data to the public project [common/quickstarts](https://app.neptune.ai/common/quickstarts) as an anonymous user.

Connect your script to Neptune application and create new run.

In [ ]:
import neptune.new as neptune

run = neptune.init_run(
    project="common/showroom",
    api_token=neptune.ANONYMOUS_API_TOKEN,
    tags=["cross-validation"],
)

Alternatively, you can log the example to your own workspace.

To do that, replace the code above with the following:

```python
from getpass import getpass

run = neptune.init_run(
    api_token=getpass("Enter your Neptune API token: "),
    project="workspace-name/project-name",  # replace with your own
)
```

For example, if your workspace name is `ml-team` and the project name is `classification`, the project argument is: `project="ml-team/classification"`.

To find your API token and project name, [log in to Neptune](https://app.neptune.ai/).
- In the top-right corner, click your avatar and select **Get your API token**.
- To find and copy your project name, navigate to the project, then click **Settings** → **Properties**.

---

You now have new run in Neptune! From here on, we'll use the `run` object to log metadata.

**To open the run in Neptune, follow the link that appeared in the cell output.**

There's not much to display yet, but keep the tab with the run open to see what happens next.

## Log config and hyperparameters

### Log Hyperparameters

In [ ]:
parameters = {
    "epochs": 2,
    "learning_rate": 1e-2,
    "batch_size": 10,
    "image_size": (3, 32, 32),
    "n_classes": 10,
    "k_folds": 2,
    "checkpoint_name": "checkpoint.pth",
    "dataset_size": 1000,
    "seed": 42,
}

In [ ]:
run["parameters"] = parameters

### Log Config
Model and Dataset

In [ ]:
import math
import torch.nn as nn


class BaseModel(nn.Module):
    def __init__(self, input_sz, hidden_dim, n_classes):
        super(BaseModel, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_sz, hidden_dim * 2),
            nn.ReLU(),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, n_classes),
        )

    def forward(self, input):
        x = input.view(-1, math.prod(parameters["image_size"]))
        return self.main(x)

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
run["parameters/device"] = device

In [ ]:
torch.manual_seed(parameters["seed"])

model = BaseModel(
    math.prod(parameters["image_size"]),
    math.prod(parameters["image_size"]),
    parameters["n_classes"],
).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=parameters["learning_rate"])

Log model, criterion and optimizer name

In [ ]:
run["parameters/model/name"] = type(model).__name__
run["parameters/model/criterion"] = type(criterion).__name__
run["parameters/model/optimizer"] = type(optimizer).__name__

In [ ]:
from torchvision import datasets, transforms

data_tfms = {
    "train": transforms.Compose(
        [
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )
}

In [ ]:
dataset = datasets.FakeData(
    size=parameters["dataset_size"],
    image_size=parameters["image_size"],
    num_classes=parameters["n_classes"],
    transform=data_tfms["train"],
)

Log dataset details

In [ ]:
run["dataset/transforms"] = data_tfms
run["dataset/size"] = parameters["dataset_size"]

## Log losses and metrics per fold 
Training Loop

In [ ]:
from sklearn.model_selection import KFold

splits = KFold(n_splits=parameters["k_folds"], shuffle=True)
epoch_acc_list, epoch_loss_list = [], []

In [ ]:
from torch.utils.data import SubsetRandomSampler, DataLoader
from tqdm.auto import tqdm, trange

for fold, (train_ids, _) in tqdm(enumerate(splits.split(dataset))):
    train_sampler = SubsetRandomSampler(train_ids)
    train_loader = DataLoader(dataset, batch_size=parameters["batch_size"], sampler=train_sampler)
    for _ in trange(parameters["epochs"]):
        epoch_acc, epoch_loss = 0, 0.0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model.forward(x)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, y)
            acc = (torch.sum(preds == y.data)) / len(x)

            # Log batch loss and acc
            run[f"fold_{fold}/training/batch/loss"].log(loss)
            run[f"fold_{fold}/training/batch/acc"].log(acc)

            loss.backward()
            optimizer.step()

        epoch_acc += torch.sum(preds == y.data).item()
        epoch_loss += loss.item() * x.size(0)

    epoch_acc_list.append((epoch_acc / len(train_loader.sampler)) * 100)
    epoch_loss_list.append(epoch_loss / len(train_loader.sampler))

    # Log model checkpoint
    torch.save(model.state_dict(), f"./{parameters['checkpoint_name']}")
    run[f"fold_{fold}/checkpoint"].upload(parameters["checkpoint_name"])
    run.sync()

In [ ]:
from statistics import mean

# log mean acc and loss
run["results/metrics/train/mean_acc"] = mean(epoch_acc_list)
run["results/metrics/train/mean_loss"] = mean(epoch_loss_list)

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()

## Explore the run in Neptune

After starting a run, you will get a link on the cell output similar to https://app.neptune.ai/o/common/org/showroom/e/SHOW-27624 with: 
* **common/showroom** replaced by **your_workspace/your_project**,
* **SHOW-27624** replaced by your *run ID*. 

**Click on the link to open the run in Neptune app.**

<img src="https://neptune.ai/wp-content/uploads/metadata-in-neptune-5.gif" alt="Drawing" style="width: 100%;"/>
<center><small>Analysing per-fold metadata</small></center>

## Cross-validation with Integrations
If you are using Neptune with XGBoost or LightGBM you can get the structure for cross-validation automatically, by using available [integrations](https://docs.neptune.ai/integrations/).
<div style="position: relative; padding-bottom: 62.5%; height: 0;"><iframe src="https://www.loom.com/embed/98dc6247c65f49b8baf7476cf996dbe4" frameborder="0" webkitallowfullscreen mozallowfullscreen allowfullscreen style="position: absolute; top: 0; left: 0; width: 100%; height: 100%;"></iframe></div>

## Conclusion

You learned how to organize your run to track cross-validation metadata with Neptune and how to present the result in the Neptune app for further comparison and analysis. 

Visit our docs for more tutorials and guides on how to use Neptune: https://docs.neptune.ai
